# Processing Training Data

I'm used to first going into EDA, but that would feel pointless without some processing done to the text data.

There are a lot of possibile things we can do to this data to prep it for future steps. Prior to getting into it here is my plan:
1. Make lowercase
2. Expand contractions
3. Noise Removal
4. Tokenization
5. Remove punctuation
6. Remove Stopwords
7. Lemmatize

## Imports and Load Data

But first, we gotta import modules and get the data in here

In [4]:
import pandas as pd
import numpy as np
!pip install contractions
import contractions #for expanding contractions to full words
import re #we'll use especially for noise removal/scrubbing
import string #use this for the punctuation feature

#nltk = Natural Language Tooklit
!pip install nltk
import nltk
from nltk.tokenize import word_tokenize 
from nltk.corpus import stopwords, wordnet
from nltk.stem import WordNetLemmatizer



[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
training_data = pd.read_csv('../../src/data/interim/training_data.csv')

training_data.info()

FileNotFoundError: [Errno 2] No such file or directory: '../../src/data/interim/training_data.csv'

In [ ]:
training_data.head()

,question,schema
0,How many heads of the departments are older th...,department_management
1,"List the name, born state and age of the heads...",department_management
2,"List the creation year, name and budget of eac...",department_management
3,What are the maximum and minimum budget of the...,department_management
4,What is the average number of employees of the...,department_management


In [ ]:
#i'm going to be adding columns to see the changes, so I'll actually swap our columns here
training_data = training_data[['schema','question']]

In [ ]:
training_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8659 entries, 0 to 8658
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   schema    8659 non-null   object
 1   question  8659 non-null   object
dtypes: object(2)
memory usage: 135.4+ KB


## Processing Work

### 1. Make Lowercase

In [ ]:
training_data['lowercase'] = training_data['question'].str.lower()

In [ ]:
training_data.head()

,schema,question,lowercase
0,department_management,How many heads of the departments are older th...,how many heads of the departments are older th...
1,department_management,"List the name, born state and age of the heads...","list the name, born state and age of the heads..."
2,department_management,"List the creation year, name and budget of eac...","list the creation year, name and budget of eac..."
3,department_management,What are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...
4,department_management,What is the average number of employees of the...,what is the average number of employees of the...


### 2. Expand Contractions

index 1443 contains "didnt'" - we can test if this works here.

In [ ]:
#breakout contractions into their full words
training_data['no_contraction'] = training_data['lowercase'].apply(lambda x: [contractions.fix(word) for word in x.split()])

#convert this to a full string again to make tokenizing more straightforward
training_data['no_contraction'] = [' '.join(map(str, l)) for l in training_data['no_contraction']]

In [ ]:
training_data.head()

,schema,question,lowercase,no_contraction
0,department_management,How many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older th...
1,department_management,"List the name, born state and age of the heads...","list the name, born state and age of the heads...","list the name, born state and age of the heads..."
2,department_management,"List the creation year, name and budget of eac...","list the creation year, name and budget of eac...","list the creation year, name and budget of eac..."
3,department_management,What are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...
4,department_management,What is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...


In [ ]:
print(training_data.iloc[1443,2])
print(training_data.iloc[1443,3])

what are the ids of instructors who didnt' teach?
what are the ids of instructors who did not' teach?


That worked! :)

### 4. Noise Removal

We don't need all that pesky punction garbase. Sorry oxford commas! You know I love you!

In [ ]:
#define "scrubbing function"

def scrub_words(text):
    """Function for some basic scrubbing courtesy of Kavita Ganesan: https://github.com/kavgan/nlp-in-practice/blob/master/text-pre-processing/Text%20Preprocessing%20Examples.ipynb"""
    
    # remove html markup
    text=re.sub("(<.*?>)","",text)
    
    #remove non-ascii and digits
    text=re.sub("(\\W|\\d)"," ",text)
    
    #remove whitespace
    text=text.strip()
    return text

In [ ]:
#apply the function to my latest column
#https://stackoverflow.com/questions/13331698/how-to-apply-a-function-to-two-columns-of-pandas-dataframe

training_data['scrubbed'] = training_data.apply(lambda x: scrub_words(x.no_contraction), axis=1)

training_data.head()

,schema,question,lowercase,no_contraction,scrubbed
0,department_management,How many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older than
1,department_management,"List the name, born state and age of the heads...","list the name, born state and age of the heads...","list the name, born state and age of the heads...",list the name born state and age of the heads...
2,department_management,"List the creation year, name and budget of eac...","list the creation year, name and budget of eac...","list the creation year, name and budget of eac...",list the creation year name and budget of eac...
3,department_management,What are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...
4,department_management,What is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...


### 4. Tokenize

With those contractions broken out now let's breakout the words for the future processing steps

In [ ]:
training_data['tokenized'] = training_data['no_contraction'].apply(word_tokenize)
training_data.head()

,schema,question,lowercase,no_contraction,scrubbed,tokenized
0,department_management,How many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older than,"[how, many, heads, of, the, departments, are, ..."
1,department_management,"List the name, born state and age of the heads...","list the name, born state and age of the heads...","list the name, born state and age of the heads...",list the name born state and age of the heads...,"[list, the, name, ,, born, state, and, age, of..."
2,department_management,"List the creation year, name and budget of eac...","list the creation year, name and budget of eac...","list the creation year, name and budget of eac...",list the creation year name and budget of eac...,"[list, the, creation, year, ,, name, and, budg..."
3,department_management,What are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,"[what, are, the, maximum, and, minimum, budget..."
4,department_management,What is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,"[what, is, the, average, number, of, employees..."


### 5. Remove Puncation

In [ ]:
punc = string.punctuation

training_data['no_punc'] = training_data['tokenized'].apply(lambda x: [word for word in x if word not in punc])

training_data.head()

,schema,question,lowercase,no_contraction,scrubbed,tokenized,no_punc
0,department_management,How many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older than,"[how, many, heads, of, the, departments, are, ...","[how, many, heads, of, the, departments, are, ..."
1,department_management,"List the name, born state and age of the heads...","list the name, born state and age of the heads...","list the name, born state and age of the heads...",list the name born state and age of the heads...,"[list, the, name, ,, born, state, and, age, of...","[list, the, name, born, state, and, age, of, t..."
2,department_management,"List the creation year, name and budget of eac...","list the creation year, name and budget of eac...","list the creation year, name and budget of eac...",list the creation year name and budget of eac...,"[list, the, creation, year, ,, name, and, budg...","[list, the, creation, year, name, and, budget,..."
3,department_management,What are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,"[what, are, the, maximum, and, minimum, budget...","[what, are, the, maximum, and, minimum, budget..."
4,department_management,What is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,"[what, is, the, average, number, of, employees...","[what, is, the, average, number, of, employees..."


### 6. Remove Stop Words

From what I understand there are multiple ways to flag which words are stop words. I'll use the standard enligh stop words in the nltk library.

In [ ]:
stop_words = set(stopwords.words('english'))
training_data['stopwords_removed'] = training_data['no_punc'].apply(lambda x: [word for word in x if word not in stop_words])

training_data.head()

,schema,question,lowercase,no_contraction,scrubbed,tokenized,no_punc,stopwords_removed
0,department_management,How many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older than,"[how, many, heads, of, the, departments, are, ...","[how, many, heads, of, the, departments, are, ...","[many, heads, departments, older, 56]"
1,department_management,"List the name, born state and age of the heads...","list the name, born state and age of the heads...","list the name, born state and age of the heads...",list the name born state and age of the heads...,"[list, the, name, ,, born, state, and, age, of...","[list, the, name, born, state, and, age, of, t...","[list, name, born, state, age, heads, departme..."
2,department_management,"List the creation year, name and budget of eac...","list the creation year, name and budget of eac...","list the creation year, name and budget of eac...",list the creation year name and budget of eac...,"[list, the, creation, year, ,, name, and, budg...","[list, the, creation, year, name, and, budget,...","[list, creation, year, name, budget, department]"
3,department_management,What are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,"[what, are, the, maximum, and, minimum, budget...","[what, are, the, maximum, and, minimum, budget...","[maximum, minimum, budget, departments]"
4,department_management,What is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,"[what, is, the, average, number, of, employees...","[what, is, the, average, number, of, employees...","[average, number, employees, departments, whos..."


### 7. Lemmatization

I've been researching these steps with some good wordpress articles, especially this one: https://towardsdatascience.com/preprocessing-text-data-using-python-576206753c28

Lemmatation has the most steps, which I'll follow here. We first apply the parts of speech with nltk. Then loop through our records and apply the wordnet equivalent POS values. This wordnet formatting is required for the lemmatizer.

In [ ]:
#add parts of speech (POS) tags to each word

training_data['pos_tags'] = training_data['stopwords_removed'].apply(nltk.tag.pos_tag)

training_data.head()

,schema,question,lowercase,no_contraction,scrubbed,tokenized,no_punc,stopwords_removed,pos_tags
0,department_management,How many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older than,"[how, many, heads, of, the, departments, are, ...","[how, many, heads, of, the, departments, are, ...","[many, heads, departments, older, 56]","[(many, JJ), (heads, NNS), (departments, NNS),..."
1,department_management,"List the name, born state and age of the heads...","list the name, born state and age of the heads...","list the name, born state and age of the heads...",list the name born state and age of the heads...,"[list, the, name, ,, born, state, and, age, of...","[list, the, name, born, state, and, age, of, t...","[list, name, born, state, age, heads, departme...","[(list, NN), (name, NN), (born, VBN), (state, ..."
2,department_management,"List the creation year, name and budget of eac...","list the creation year, name and budget of eac...","list the creation year, name and budget of eac...",list the creation year name and budget of eac...,"[list, the, creation, year, ,, name, and, budg...","[list, the, creation, year, name, and, budget,...","[list, creation, year, name, budget, department]","[(list, NN), (creation, NN), (year, NN), (name..."
3,department_management,What are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,"[what, are, the, maximum, and, minimum, budget...","[what, are, the, maximum, and, minimum, budget...","[maximum, minimum, budget, departments]","[(maximum, JJ), (minimum, JJ), (budget, NN), (..."
4,department_management,What is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,"[what, is, the, average, number, of, employees...","[what, is, the, average, number, of, employees...","[average, number, employees, departments, whos...","[(average, JJ), (number, NN), (employees, NNS)..."


In [ ]:
#define function to convert to wordnet
def get_wordnet_pos(tag):
    """to be used when going through the pos_tags to conver to wordnet format necessary for lemmatizing"""
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

In [ ]:
#use function to apply wordnet formatting to our tokenized data
training_data['wordnet_pos'] = training_data['pos_tags'].apply(lambda x: [(word, get_wordnet_pos(pos_tag)) for (word, pos_tag) in x])

training_data.head()

,schema,question,lowercase,no_contraction,scrubbed,tokenized,no_punc,stopwords_removed,pos_tags,wordnet_pos
0,department_management,How many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older than,"[how, many, heads, of, the, departments, are, ...","[how, many, heads, of, the, departments, are, ...","[many, heads, departments, older, 56]","[(many, JJ), (heads, NNS), (departments, NNS),...","[(many, a), (heads, n), (departments, n), (old..."
1,department_management,"List the name, born state and age of the heads...","list the name, born state and age of the heads...","list the name, born state and age of the heads...",list the name born state and age of the heads...,"[list, the, name, ,, born, state, and, age, of...","[list, the, name, born, state, and, age, of, t...","[list, name, born, state, age, heads, departme...","[(list, NN), (name, NN), (born, VBN), (state, ...","[(list, n), (name, n), (born, v), (state, n), ..."
2,department_management,"List the creation year, name and budget of eac...","list the creation year, name and budget of eac...","list the creation year, name and budget of eac...",list the creation year name and budget of eac...,"[list, the, creation, year, ,, name, and, budg...","[list, the, creation, year, name, and, budget,...","[list, creation, year, name, budget, department]","[(list, NN), (creation, NN), (year, NN), (name...","[(list, n), (creation, n), (year, n), (name, n..."
3,department_management,What are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,"[what, are, the, maximum, and, minimum, budget...","[what, are, the, maximum, and, minimum, budget...","[maximum, minimum, budget, departments]","[(maximum, JJ), (minimum, JJ), (budget, NN), (...","[(maximum, a), (minimum, a), (budget, n), (dep..."
4,department_management,What is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,"[what, is, the, average, number, of, employees...","[what, is, the, average, number, of, employees...","[average, number, employees, departments, whos...","[(average, JJ), (number, NN), (employees, NNS)...","[(average, a), (number, n), (employees, n), (d..."


In [ ]:
#finally apply the lemmatizer, loopoing through our dataframe using the word and wordnet tag
wnl = WordNetLemmatizer() #initialize lemmatizer

training_data['lemmatized'] = training_data['wordnet_pos'].apply(lambda x: [wnl.lemmatize(word, tag) for word, tag in x])

training_data.head()

,schema,question,lowercase,no_contraction,scrubbed,tokenized,no_punc,stopwords_removed,pos_tags,wordnet_pos,lemmatized
0,department_management,How many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older th...,how many heads of the departments are older than,"[how, many, heads, of, the, departments, are, ...","[how, many, heads, of, the, departments, are, ...","[many, heads, departments, older, 56]","[(many, JJ), (heads, NNS), (departments, NNS),...","[(many, a), (heads, n), (departments, n), (old...","[many, head, department, older, 56]"
1,department_management,"List the name, born state and age of the heads...","list the name, born state and age of the heads...","list the name, born state and age of the heads...",list the name born state and age of the heads...,"[list, the, name, ,, born, state, and, age, of...","[list, the, name, born, state, and, age, of, t...","[list, name, born, state, age, heads, departme...","[(list, NN), (name, NN), (born, VBN), (state, ...","[(list, n), (name, n), (born, v), (state, n), ...","[list, name, bear, state, age, head, departmen..."
2,department_management,"List the creation year, name and budget of eac...","list the creation year, name and budget of eac...","list the creation year, name and budget of eac...",list the creation year name and budget of eac...,"[list, the, creation, year, ,, name, and, budg...","[list, the, creation, year, name, and, budget,...","[list, creation, year, name, budget, department]","[(list, NN), (creation, NN), (year, NN), (name...","[(list, n), (creation, n), (year, n), (name, n...","[list, creation, year, name, budget, department]"
3,department_management,What are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,what are the maximum and minimum budget of the...,"[what, are, the, maximum, and, minimum, budget...","[what, are, the, maximum, and, minimum, budget...","[maximum, minimum, budget, departments]","[(maximum, JJ), (minimum, JJ), (budget, NN), (...","[(maximum, a), (minimum, a), (budget, n), (dep...","[maximum, minimum, budget, department]"
4,department_management,What is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,what is the average number of employees of the...,"[what, is, the, average, number, of, employees...","[what, is, the, average, number, of, employees...","[average, number, employees, departments, whos...","[(average, JJ), (number, NN), (employees, NNS)...","[(average, a), (number, n), (employees, n), (d...","[average, number, employee, department, whose,..."


That is all the steps I had in mind. Let's create a final dataframe with only our schema and fully processed data.

## Create and Save Processed Training Data

In [ ]:
training_proc = training_data[['schema','lemmatized']].rename(columns={'lemmatized': 'question_proc'})

training_proc.head()

,schema,question_proc
0,department_management,"[many, head, department, older, 56]"
1,department_management,"[list, name, bear, state, age, head, departmen..."
2,department_management,"[list, creation, year, name, budget, department]"
3,department_management,"[maximum, minimum, budget, department]"
4,department_management,"[average, number, employee, department, whose,..."


In [ ]:
#export csv
#filepath = '../data/interim/processed_training_data.csv'
#training_proc.to_csv(filepath, index=False)

In [ ]:
#export as pickle file
#filepath = '../data/interim/processed_training_data.pkl'
#training_proc.to_pickle(filepath)